# __Encoder/Decoder - Tensorboard__

### __Deep Learning__

#### __Project: Image Captioning with Visual Attention__

In [1]:
import os
import shutil
os.chdir(os.environ["PYTHONPATH"])

import torch
import torchvision
from torch.utils import tensorboard, data

import scripts.data_loading as dl
import scripts.data_processing as dp
import scripts.model as mdl

%load_ext autoreload
%autoreload 2

In [2]:
ENCODER_OUTPUT_LOG_DIR = os.path.join("runs", "ENCODER_MODEL")
DECODER_OUTPUT_LOG_DIR = os.path.join("runs", "DECODER_MODEL")

if os.path.exists(ENCODER_OUTPUT_LOG_DIR):
    shutil.rmtree(ENCODER_OUTPUT_LOG_DIR)
    
if os.path.exists(DECODER_OUTPUT_LOG_DIR):
    shutil.rmtree(DECODER_OUTPUT_LOG_DIR)

In [3]:
coco = dl.CocoCaptions(
    dl.TRAINING_DATASET_PATHS[dl.DatasetType.TRAIN], 
    transform=dp.VGGNET_PREPROCESSING_PIPELINE, 
    target_transform=dp.TextPipeline()
)

data_loader = dl.CocoLoader(coco, batch_size=1, num_workers=2)
batch = next(iter(data_loader))

loading annotations into memory...
Done (t=2.12s)
creating index...
index created!


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
torch.cuda.get_device_name(torch.cuda.current_device())

'GeForce GTX 960M'

In [6]:
images, captions = batch[0].to(device), batch[1].to(device)

encoder = mdl.VGG19Encoder()
encoder.to(device)

encoder_tensorboard = tensorboard.SummaryWriter(log_dir=ENCODER_OUTPUT_LOG_DIR)
encoder_tensorboard.add_graph(encoder, input_to_model=images)
encoder_tensorboard.close()

In [7]:
decoder = mdl.LSTMDecoder(
    num_embeddings=len(coco.target_transform.vocabulary),
    embedding_dim=64,
    encoder_dim=196,
    decoder_dim=128,
    attention_dim=256,
)
decoder.to(device)

decoder_tensorboard = tensorboard.SummaryWriter(log_dir=DECODER_OUTPUT_LOG_DIR)
decoder_tensorboard.add_graph(decoder, input_to_model=[*encoder(images), captions])
decoder_tensorboard.close()

/home/wiktorlazarski/Desktop/Image-Captioning-with-Visual-Attention/scripts/model.py:148: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  for time_step in range(caption_len - 1):
